This Keras tutorial will show you how to build a CNN to achieve >99% accuracy with the MNIST dataset

![title](images/CNN-example-block-diagram.jpg)

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load image data from MNIST.

MNIST is a great dataset for getting started with deep learning and computer vision. It's a big enough challenge to warrant neural networks, but it's manageable on a single computer. 

The Keras library conveniently includes it already. We can load it like so:

In [2]:
batch_size = 128
num_classes = 10
epochs = 10

In [3]:
# input image dimensions
img_x, img_y = 28, 28
input_shape = (img_x, img_y, 1)

# load the MNIST data set, which already splits into train and test sets for us
(X_train, y_train), (X_test, y_test) = mnist.load_data()

We can look at the shape of the dataset:

In [4]:
X_train.shape

(60000, 28, 28)

### Preprocess input data for Keras.

Our MNIST images only have a depth of 1, but we must explicitly declare that.

In other words, we want to transform our dataset from having shape (n, width, height) to (n, depth, width, height).

Here's how we can do that easily:

In [18]:
X_train = X_train.reshape(X_train.shape[0], img_x, img_y, 1)
X_test = X_test.reshape(X_test.shape[0], img_x, img_y, 1)

To confirm, we can print X_train's dimensions again:

In [19]:
X_train.shape

(60000, 28, 28, 1)

The final preprocessing step for the input data is to convert our data type to float32 and normalize our data values to the range [0, 1].

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Now, our input data are ready for model training.

Preprocess class labels for Keras.

Next, let's take a look at the shape of our class label data:

In [6]:
y_train.shape

(60000,)

Hmm... that may be problematic. We should have 10 different classes, one for each digit, but it looks like we only have a 1-dimensional array. Let's take a look at the labels for the first 10 training samples:

In [7]:
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

And there's the problem. The y_train and y_test data are not split into 10 distinct class labels, but rather are represented as a single array with the class values.

We can fix this easily:

In [8]:
Y_train = keras.utils.to_categorical(y_train, num_classes)
Y_test = keras.utils.to_categorical(y_test, num_classes)

Now we can take another look:

In [9]:
Y_train.shape

(60000, 10)

There we go... much better!

### Define model architecture.

In [10]:
model = Sequential()

Models in Keras can come in two forms – Sequential and via the Functional API.  

For most deep learning networks that you build, the Sequential model is likely what you will use.  It allows you to easily stack sequential layers (and even recurrent layers) of the network in order from input to output.  The functional API allows you to build more complicated architectures.

The first line declares the model type as Sequential().

In [11]:
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))

Next, we add a 2D convolutional layer to process the 2D MNIST input images.  The first argument passed to the Conv2D() layer function is the number of output channels – in this case we have 32 output channels (as per the architecture shown at the beginning).  The next input is the kernel_size, which in this case we have chosen to be a 5×5 moving window, followed by the strides in the x and y directions (1, 1).  

Next, the activation function is a rectified linear unit and finally we have to supply the model with the size of the input to the layer (which is declared in another part of the code – see here).  Declaring the input shape is only required of the first layer – Keras is good enough to work out the size of the tensors flowing through the model from there.

Also notice that we don’t have to declare any weights or bias variables like we do in TensorFlow, Keras sorts that out for us.

In [12]:
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

Next we add a 2D max pooling layer.  The definition of the layer is dead easy.  We simply specify the size of the pooling in the x and y directions – (2, 2) in this case, and the strides.  That’s it.

In [13]:
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

Next we add another convolutional + max pooling layer, with 64 output channels.  The default strides argument in the Conv2D() function is (1, 1) in Keras, so we can leave it out.  The default strides argument in Keras is to make it equal ot the pool size, so again, we can leave it out.

The input tensor for this layer is (batch_size, 28, 28,  32) – the 28 x 28 is the size of the image, and the 32 is the number of output channels from the previous layer.  However, notice we don’t have to explicitly detail what the shape of the input is – Keras will work it out for us.  This allows rapid assembling of network architectures without having to worry too much about the sizes of the tensors flowing around our networks.

In [14]:
model.add(Flatten())


model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


Now that we’ve built our convolutional layers in this Keras tutorial, we want to flatten the output from these to enter our fully connected layers.

In TensorFlow, we had to figure out what the size of our output tensor from the convolutional layers was in order to flatten it, and also to determine explicitly the size of our weight and bias variables.  Sure, this isn’t too difficult – but it just makes our life easier not to have to think about it too much.

The next two lines declare our fully connected layers – using the Dense() layer in Keras.  Again, it is very simple.  First we specify the size – in line with our architecture, we specify 1000 nodes, each activated by a ReLU function.  The second is our soft-max classification, or output layer, which is the size of the number of our classes (10 in this case, for our 10 possible hand-written digits).

That’s it – we have successfully developed the architecture of our CNN in only 8 lines.  Now let’s see what we have to do to train the model and perform predictions.

### Training and evaluating our convolutional neural network

We have now developed the architecture of the CNN in Keras, but we haven’t specified the loss function, or told the framework what type of optimiser to use (i.e. gradient descent, Adam optimiser etc.).  In Keras, this can be performed in one command:

In [15]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Keras supplies many loss functions (or you can build your own) as can be seen here.  In this case, we will use the standard cross entropy for categorical class classification (keras.losses.categorical_crossentropy).  

we’ll use the Adam optimizer (keras.optimizers.Adam) as we did in the CNN TensorFlow tutorial.  Finally, we can specify a metric that will be calculated when we run evaluate() on the model. 

In TensorFlow we would have to define an accuracy calculating operation which we would need to call in order to assess the accuracy.  In this case, Keras makes it easy for us.  See here for a list of metrics that can be used.

Next, we want to train our model.  This can be done by again running a single command in Keras:

In [20]:
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 118s 2ms/step - loss: 14.6607 - acc: 0.0903 - val_loss: 14.6804 - val_acc: 0.0892
Epoch 2/10
60000/60000 [==============================] - 127s 2ms/step - loss: 14.6618 - acc: 0.0904 - val_loss: 14.6804 - val_acc: 0.0892
Epoch 3/10
60000/60000 [==============================] - 152s 3ms/step - loss: 14.6618 - acc: 0.0903 - val_loss: 14.6804 - val_acc: 0.0892
Epoch 4/10
60000/60000 [==============================] - 102s 2ms/step - loss: 14.6618 - acc: 0.0903 - val_loss: 14.6804 - val_acc: 0.0892
Epoch 5/10
60000/60000 [==============================] - 101s 2ms/step - loss: 14.6618 - acc: 0.0903 - val_loss: 14.6804 - val_acc: 0.0892
Epoch 6/10
60000/60000 [==============================] - 103s 2ms/step - loss: 14.6618 - acc: 0.0903 - val_loss: 14.6804 - val_acc: 0.0892
Epoch 7/10
60000/60000 [==============================] - 103s 2ms/step - loss: 14.6618 - acc: 0.0903 - val_lo

Finally, we pass the validation or test data to the fit function so Keras knows what data to test the metric against when evaluate() is run on the model.  Ignore the callbacks argument for the moment – that will be discussed shortly.

Once the model is trained, we can then evaluate it and print the results:

In [22]:
score = model.evaluate(X_test, Y_test, verbose=0)


print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 14.680361064147949
Test accuracy: 0.0892


After 10 epochs of training the above model, we achieve an accuracy of 99.2%, which is the same as what we achieved in TensorFlow for the same network. 